In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pesq import pesq
import pystoi

from utils import performance, calculate_snr, reconstruct_signal_from_stft
from data_generators import NoisyTargetWithMetricsGenerator
from artigos import AttentionResUnet
from sound import Sound
from kalman import kalman
from tqdm import tqdm
import time

from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K

from wiener import wiener_filter

In [3]:
from artigos.PRIDNet import Convolutional_block, Channel_attention, AttResUnetConvLSTM, Avg_pool_Unet_Upsample_msfe, Multi_scale_feature_extraction, Kernel_selecting_module
from keras.utils import custom_object_scope

In [4]:
from artigos.DeepNoiseSuppressionWithNonIntrusivePESQNet import weighted_msle

In [14]:
base_shape_size = 8192*4
ws = 255
ol = 128

In [6]:
# att_unet_path = '/tf/Etapa 3/Artigos/Attention ResU-Net/AttentionResUnet_checkpoints/attention_resunet_epochs_53_loss_mse_0.0106_vallos_0.0112.h5'
att_unet_path = '/tf/Etapa 3/Artigos/Attention ResU-Net/AttentionResUnet_checkpoints/1_mirror_attention_res_unet_custom-mlse.h5'
# att_unet_path = '/tf/Etapa 3/Artigos/Attention ResU-Net/checkpoint-2024-04-21_04-20-09.h5'
fcrn_path = '/tf/Etapa 3/Artigos/Non Intrusive PESQNet/DNS_checkpoints/best_DNS_128x64.h5'
pridnet_path = '/tf/Etapa 3/Artigos/PRIDNet/PRIDNet_loss-0.0165-epochs-54.h5'
# pridnet_path = '/tf/Etapa 3/Artigos/PRIDNet/best_PRIDNet_0.0031_epochs_92.h5'

In [7]:
# unet = load_model(att_unet_path)

# with custom_object_scope({'weighted_msle': weighted_msle}):
#     fcrn = load_model(fcrn_path)

# with custom_object_scope({'Convolutional_block': Convolutional_block,
#                           'Channel_attention': Channel_attention,
#                           'AttResUnetConvLSTM': AttResUnetConvLSTM,
#                           'Avg_pool_Unet_Upsample_msfe': Avg_pool_Unet_Upsample_msfe,
#                           'Multi_scale_feature_extraction': Multi_scale_feature_extraction,
#                           'Kernel_selecting_module': Kernel_selecting_module}):
#     pridnet = load_model(pridnet_path)

In [8]:
# Carrega o unet na GPU 0
with tf.device('/GPU:0'):
    unet = load_model(att_unet_path)

# Carrega o fcrn na GPU 1
with tf.device('/GPU:1'):
    with custom_object_scope({'weighted_msle': weighted_msle}):
        fcrn = load_model(fcrn_path)

# Carrega o pridnet na GPU 0
with tf.device('/GPU:0'):
    with custom_object_scope({'Convolutional_block': Convolutional_block,
                              'Channel_attention': Channel_attention,
                              'AttResUnetConvLSTM': AttResUnetConvLSTM,
                              'Avg_pool_Unet_Upsample_msfe': Avg_pool_Unet_Upsample_msfe,
                              'Multi_scale_feature_extraction': Multi_scale_feature_extraction,
                              'Kernel_selecting_module': Kernel_selecting_module}):
        pridnet = load_model(pridnet_path)

In [15]:
sound_base = Sound('/tf/Dados/Vozes/train/', '/tf/Dados/Ruido/train/', base_shape_size)

Loading Speech Files: 100%|██████████| 5725/5725 [00:03<00:00, 1607.27it/s]


In [16]:
data_generator = NoisyTargetWithMetricsGenerator(sound_base.clean_sounds, sound_base.noise_sounds)

In [17]:
import soundfile as sf
import os

# Função para salvar o áudio com as métricas no nome
def save_audio_with_metrics(audio, filename, folder, SNR=None, PESQ=None, STOI=None):
    metrics = []
    if SNR is not None:
        metrics.append(f"SNR_{SNR:.2f}")
    if PESQ is not None:
        metrics.append(f"PESQ_{PESQ:.2f}")
    if STOI is not None:
        metrics.append(f"STOI_{STOI:.2f}")
    
    metrics_str = "_".join(metrics)
    if metrics_str:
        filename = f"{filename}_{metrics_str}.wav"
    else:
        filename = f"{filename}.wav"
    
    file_path = os.path.join(folder, filename)
    sf.write(file_path, audio, samplerate=8000)
    print(f"Saved: {file_path}")

In [18]:
base_folder = "/tf/Resultado - Sons Filtrados 3"

# Criar diretório se não existir
os.makedirs(base_folder, exist_ok=True)

In [19]:
batch_num = 1

In [20]:
window_size = 600
order = 11
df_resultado = pd.DataFrame()
batch_size=8

# Acumulador para os tempos de execução
wiener_execution_times = []
kalman_execution_times = []
att_execution_times = []
fcrn_execution_times = []
pridnet_execution_times = []

In [21]:
it = 1
while df_resultado.shape[0] < 8:
    # try:
        for _ in tqdm(range(batch_num)):
            (x_wave_batch, y_wave_batch), (x_stft_batch, y_stft_batch), metrics_batch_df = next(data_generator.generate_sample_completo(batch_size=batch_size))
        
            ## Filtragem dos sinais
            
            # Aplica o filtro de Wiener aos sinais ruidosos
            wiener_start_time = time.time()  # Início da medição de tempo
            wiener_batch = [wiener_filter(noisy_signal, snr) for noisy_signal, snr in zip(x_wave_batch, metrics_batch_df['SNR'])]
            wiener_end_time = time.time()  # Fim da medição de tempo
            wiener_execution_times.append((wiener_end_time - wiener_start_time) / batch_size)
        
            # Aplica o filtro de Kalman aos sinais ruidosos
            kalman_start_time = time.time()  # Início da medição de tempo
            kalman_batch = [kalman(noisy_signal, 600, 11, 8000, snr) for noisy_signal, snr in zip(x_wave_batch, metrics_batch_df['SNR'])]
            kalman_end_time = time.time()  # Fim da medição de tempo
            kalman_execution_times.append((kalman_end_time - kalman_start_time) / (batch_size * 6))
        
            # Aplica o filtro de Attention Res U-Net aos sinais ruidosos
            att_batch = []

            freqs_n = 128 # 128
            times_n = 256 # 64
            model_times = 64
            time_limit = 32640 # 8064
        
            for noisy_signal in x_stft_batch:
                # Dividir o sinal em 4 segmentos de (128, 64, 1), criando o batch
                x = noisy_signal[..., 0]
                x_segments = x.reshape(freqs_n, int(times_n / model_times), model_times, 1).transpose(1, 0, 2, 3)  # Shape final: (4, 128, 64, 1)
                
                segment_outputs = []
                for x in x_segments:
                    # Passar cada segmento pelo modelo
                    att_start_time = time.time()
                    F_f = unet.predict(x[np.newaxis, ...], verbose=False)  # Adiciona batch de tamanho 1
                    att_end_time = time.time()
                    att_execution_times.append(att_end_time - att_start_time)
                    
                    # Remover a dimensão de batch e armazenar a saída
                    segment_outputs.append(F_f[0])  # Shape de F_f[0] é (128, 64, 1)
            
                # Concatenar os segmentos ao longo da segunda dimensão (64 * 4 = 256)
                F_f = np.concatenate(segment_outputs, axis=1)  # Shape final: (128, 256, 1)

                A_f = F_f.reshape(freqs_n, times_n)
                phi_n = (noisy_signal[..., 1].reshape(freqs_n, times_n) - 0.5) * 2 * np.pi
        
                s_f = reconstruct_signal_from_stft(A_f, phi_n)
                
                att_batch.append(s_f)
                ###################################################
                # x = noisy_signal[np.newaxis, ...]
                
                # # print(f'Noisy signal shape: {noisy_signal.shape}')
                # # print(f'X shape: {x.shape}')
                
                # att_start_time = time.time()  # Início da medição de tempo
                # F_f = unet.predict(x, verbose=False)
                # att_end_time = time.time()  # Fim da medição de tempo
                # att_execution_times.append(att_end_time - att_start_time)
                
                # # print(f'NN output shape: {F_f.shape}')
                # F_f = F_f.reshape(128, 64, 2)
                
                # A_f = F_f[:, :, 0]
                # phi_n = (F_f[:, :, 1]- 0.5) * 2 * np.pi
        
                # s_f = reconstruct_signal_from_stft(A_f, phi_n)
                # # print(f'Shape sinal final: {s_f.shape}')
                # att_batch.append(s_f)
        
            # Aplica o filtro de FCRN aos sinais ruidosos
            fcrn_batch = []
        
            for noisy_signal in x_stft_batch:
                x = noisy_signal[..., 0]
                x_segments = x.reshape(freqs_n, int(times_n / model_times), model_times, 1).transpose(1, 0, 2, 3)  # Shape final: (4, 128, 64, 1)
                
                segment_outputs = []
                for x in x_segments:
                    fcrn_start_time = time.time()  # Início da medição de tempo
                    F_f = fcrn.predict(x[np.newaxis, ...], verbose=False)
                    fcrn_end_time = time.time()  # Fim da medição de tempo
                    fcrn_execution_times.append(fcrn_end_time - fcrn_start_time)

                    segment_outputs.append(F_f[0])
                # print(f'NN output shape: {F_f.shape}')
                # Concatenar os segmentos ao longo da segunda dimensão (64 * 4 = 256)
                F_f = np.concatenate(segment_outputs, axis=1)  # Shape final: (128, 256, 1)
        
                A_f = F_f.reshape(freqs_n, times_n)
                phi_n = (noisy_signal[..., 1].reshape(freqs_n, times_n) - 0.5) * 2 * np.pi
        
                s_f = reconstruct_signal_from_stft(A_f, phi_n)
                # print(f'Shape sinal final: {s_f.shape}')
                fcrn_batch.append(s_f)
        
            # Aplica o filtro de PRIDNet aos sinais ruidosos
            pridnet_batch = []
        
            for noisy_signal in x_stft_batch:
                # x = noisy_signal[np.newaxis, ...]
                x_segments = noisy_signal.reshape(freqs_n, int(times_n / model_times), model_times, 2).transpose(1, 0, 2, 3)

                segment_outputs = []
                for x in x_segments:
                    pridnet_start_time = time.time()  # Início da medição de tempo
                    F_f = pridnet.predict(x[np.newaxis, ...], verbose=False)
                    pridnet_end_time = time.time()  # Fim da medição de tempo
                    pridnet_execution_times.append(pridnet_end_time - pridnet_start_time)
                    
                    segment_outputs.append(F_f[0])
                
                # print(f'NN output shape: {F_f.shape}')
                F_f = np.concatenate(segment_outputs, axis=1)
                F_f = F_f.reshape(freqs_n, times_n, 2)
                
                A_f = F_f[...,  0]
                phi_n = (F_f[..., 1]- 0.5) * 2 * np.pi
        
                s_f = reconstruct_signal_from_stft(A_f, phi_n)
                # print(f'Shape sinal final: {s_f.shape}')
                pridnet_batch.append(s_f)
        
        
            
            #Calculo das metricas
            
            # Wiener Calcula as métricas para os sinais filtrados
            wiener_pesq_scores = [pesq(8000, clean.reshape(-1), filtered.reshape(-1), 'nb') for clean, filtered in zip(y_wave_batch, wiener_batch)]
            wiener_stoi_scores = [pystoi.stoi(clean, filtered, 8000) for clean, filtered in zip(y_wave_batch, wiener_batch)]
            wiener_snr_scores = [calculate_snr(clean, filtered) for clean, filtered in zip(y_wave_batch, wiener_batch)]
        
            # Kalman Calcula as métricas para os sinais filtrados
            kalman_pesq_scores = [pesq(8000, clean.reshape(-1), filtered.reshape(-1), 'nb') for clean, filtered in zip(y_wave_batch, kalman_batch)]
            kalman_stoi_scores = [pystoi.stoi(clean, filtered, 8000) for clean, filtered in zip(y_wave_batch, kalman_batch)]
            kalman_snr_scores = [calculate_snr(clean, filtered) for clean, filtered in zip(y_wave_batch, kalman_batch)]
        
            # Att Calcula as métricas para os sinais filtrados
            att_pesq_scores = [pesq(8000, clean[:time_limit].reshape(-1), filtered.reshape(-1), 'nb') for clean, filtered in zip(y_wave_batch, att_batch)]
            att_stoi_scores = [pystoi.stoi(clean[:time_limit], filtered, 8000) for clean, filtered in zip(y_wave_batch, att_batch)]
            att_snr_scores = [calculate_snr(clean[:time_limit], filtered) for clean, filtered in zip(y_wave_batch, att_batch)]
        
            # FCRN Calcula as métricas para os sinais filtrados
            fcrn_pesq_scores = [pesq(8000, clean[:time_limit].reshape(-1), filtered.reshape(-1), 'nb') for clean, filtered in zip(y_wave_batch, fcrn_batch)]
            fcrn_stoi_scores = [pystoi.stoi(clean[:time_limit], filtered, 8000) for clean, filtered in zip(y_wave_batch, fcrn_batch)]
            fcrn_snr_scores = [calculate_snr(clean[:time_limit], filtered) for clean, filtered in zip(y_wave_batch, fcrn_batch)]
        
            # PRIDNet Calcula as métricas para os sinais filtrados
            pridnet_pesq_scores = [pesq(8000, clean[:time_limit].reshape(-1), filtered.reshape(-1), 'nb') for clean, filtered in zip(y_wave_batch, pridnet_batch)]
            pridnet_stoi_scores = [pystoi.stoi(clean[:time_limit], filtered, 8000) for clean, filtered in zip(y_wave_batch, pridnet_batch)]
            pridnet_snr_scores = [calculate_snr(clean[:time_limit], filtered) for clean, filtered in zip(y_wave_batch, pridnet_batch)]
        
        
            
            # Juntar no dataframe
            
            # Wiener Adiciona as métricas dos sinais filtrados ao DataFrame
            metrics_batch_df['PESQ (Wiener)'] = wiener_pesq_scores
            metrics_batch_df['STOI (Wiener)'] = wiener_stoi_scores
            metrics_batch_df['SNR (Wiener)'] = wiener_snr_scores
        
            # Kalman Adiciona as métricas dos sinais filtrados ao DataFrame
            metrics_batch_df['PESQ (Kalman)'] = kalman_pesq_scores
            metrics_batch_df['STOI (Kalman)'] = kalman_stoi_scores
            metrics_batch_df['SNR (Kalman)'] = kalman_snr_scores
        
            # Att Adiciona as métricas dos sinais filtrados ao DataFrame
            metrics_batch_df['PESQ (Attention Res U-Net)'] = att_pesq_scores
            metrics_batch_df['STOI (Attention Res U-Net)'] = att_stoi_scores
            metrics_batch_df['SNR (Attention Res U-Net)'] = att_snr_scores
        
            # FCRN Adiciona as métricas dos sinais filtrados ao DataFrame
            metrics_batch_df['PESQ (FCRN)'] = fcrn_pesq_scores
            metrics_batch_df['STOI (FCRN)'] = fcrn_stoi_scores
            metrics_batch_df['SNR (FCRN)'] = fcrn_snr_scores
        
            # PRIDNet Adiciona as métricas dos sinais filtrados ao DataFrame
            metrics_batch_df['PESQ (PRIDNet)'] = pridnet_pesq_scores
            metrics_batch_df['STOI (PRIDNet)'] = pridnet_stoi_scores
            metrics_batch_df['SNR (PRIDNet)'] = pridnet_snr_scores
        
            df_resultado = pd.concat([df_resultado, metrics_batch_df], ignore_index=True)

            # Salvar sons
            sound_name = "som_"+str(it)
            
            # Salvar arquivos de áudio
            save_audio_with_metrics(y_wave_batch[0], f"{sound_name}_limpo", base_folder)
            save_audio_with_metrics(x_wave_batch[0], f"{sound_name}_ruidoso", base_folder, SNR=metrics_batch_df['SNR'][0], PESQ=metrics_batch_df['PESQ'][0], STOI=metrics_batch_df['STOI'][0])
            save_audio_with_metrics(wiener_batch[0], f"{sound_name}_wiener", base_folder, SNR=wiener_snr_scores[0], PESQ=wiener_pesq_scores[0], STOI=wiener_stoi_scores[0])
            save_audio_with_metrics(kalman_batch[0], f"{sound_name}_kalman", base_folder, SNR=kalman_snr_scores[0], PESQ=kalman_pesq_scores[0], STOI=kalman_stoi_scores[0])
            save_audio_with_metrics(att_batch[0], f"{sound_name}_attresunet", base_folder, SNR=att_snr_scores[0], PESQ=att_pesq_scores[0], STOI=att_stoi_scores[0])
            save_audio_with_metrics(fcrn_batch[0], f"{sound_name}_fcrn", base_folder, SNR=fcrn_snr_scores[0], PESQ=fcrn_pesq_scores[0], STOI=fcrn_stoi_scores[0])
            save_audio_with_metrics(pridnet_batch[0], f"{sound_name}_pridnet", base_folder, SNR=pridnet_snr_scores[0], PESQ=pridnet_pesq_scores[0], STOI=pridnet_stoi_scores[0])

            it += 1
    # except:
    #     continue

  0%|          | 0/1 [00:00<?, ?it/s]

Saved: /tf/Resultado - Sons Filtrados 3/som_1_limpo.wav
Saved: /tf/Resultado - Sons Filtrados 3/som_1_ruidoso_SNR_0.71_PESQ_1.42_STOI_0.63.wav
Saved: /tf/Resultado - Sons Filtrados 3/som_1_wiener_SNR_4.98_PESQ_1.29_STOI_0.60.wav
Saved: /tf/Resultado - Sons Filtrados 3/som_1_kalman_SNR_1.48_PESQ_1.49_STOI_0.63.wav


100%|██████████| 1/1 [00:17<00:00, 17.40s/it]

Saved: /tf/Resultado - Sons Filtrados 3/som_1_attresunet_SNR_7.90_PESQ_2.13_STOI_0.79.wav
Saved: /tf/Resultado - Sons Filtrados 3/som_1_fcrn_SNR_5.35_PESQ_1.59_STOI_0.72.wav
Saved: /tf/Resultado - Sons Filtrados 3/som_1_pridnet_SNR_2.73_PESQ_1.81_STOI_0.70.wav


In [22]:
x[np.newaxis, ..., 0, np.newaxis,].shape

(1, 128, 64, 1)

In [23]:
from IPython.display import Audio
Audio(np.array(att_batch)[0], rate=8000)

In [24]:
from IPython.display import Audio
Audio(np.array(fcrn_batch)[0], rate=8000)

In [26]:
from IPython.display import Audio
Audio(np.array(pridnet_batch)[0], rate=8000)

In [ ]:
df_resultado.shape

In [64]:
w = np.array(wiener_execution_times)
k = np.array(kalman_execution_times)
at = np.array(att_execution_times)
fc = np.array(fcrn_execution_times)
prid = np.array(pridnet_execution_times)

In [65]:
print(f'Wiener {w.shape}')
print(f'Kalman {k.shape}')
print(f'Atte {at.shape}')
print(f'FCRN {fc.shape}')
print(f'PRID {prid.shape}')

Wiener (1,)
Kalman (1,)
Atte (512,)
FCRN (512,)
PRID (512,)


In [66]:
print(f'Wiener {w.mean()*1e03}')
print(f'Kalman {k.mean()*1e03}')
print(f'Atte {at.mean()*1e03}')
print(f'FCRN {fc.mean()*1e03}')
print(f'PRID {prid.mean()*1e03}')

Wiener 0.5399417132139206
Kalman 45.962437676886715
Atte 26.44268050789833
FCRN 99.05924508348107
PRID 42.09368536248803


In [67]:
df_resultado

,PESQ,STOI,SNR,PESQ (Wiener),STOI (Wiener),SNR (Wiener),PESQ (Kalman),STOI (Kalman),SNR (Kalman),PESQ (Attention Res U-Net),STOI (Attention Res U-Net),SNR (Attention Res U-Net),PESQ (FCRN),STOI (FCRN),SNR (FCRN),PESQ (PRIDNet),STOI (PRIDNet),SNR (PRIDNet)
0,1.834639,0.922536,13.858974,1.853474,0.910377,15.449069,1.904644,0.925980,14.483804,2.674830,0.947738,11.765833,2.648618,0.950704,11.786112,2.167312,0.874286,3.157265
1,2.349454,0.739440,9.416807,2.512511,0.726105,9.536754,2.534724,0.750023,9.621022,2.700220,0.745462,10.742700,2.609330,0.734612,8.680060,2.437476,0.674168,1.972426
2,1.731145,0.804626,12.818114,1.720713,0.832290,14.390564,1.806928,0.817762,13.726444,2.367061,0.847716,13.355176,2.321798,0.828684,13.385630,1.920905,0.746245,3.367212
3,1.818251,0.884798,10.316397,1.762743,0.835711,10.799181,1.929399,0.891028,10.688582,2.761566,0.924826,12.456983,2.501744,0.919539,11.787403,2.172186,0.824208,2.774820
4,1.920977,0.837411,6.893727,1.695382,0.829436,8.641363,2.043260,0.833839,7.578144,2.609229,0.884256,10.175494,2.450127,0.883049,10.104828,2.329055,0.842705,3.368013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2.379673,0.915459,14.353331,2.210690,0.902156,15.406433,2.422063,0.921373,14.685705,2.808851,0.939695,11.042832,2.653256,0.933280,10.670376,2.344525,0.842316,2.708364
124,2.537612,0.791952,0.939196,2.442170,0.781978,0.399186,2.960601,0.834839,1.593486,3.405750,0.847218,11.285430,3.394133,0.845995,8.341354,2.538458,0.785245,3.361079
125,1.196253,0.683401,-0.013146,1.236647,0.655590,3.129707,1.263955,0.700077,3.844749,1.558901,0.749910,5.036595,1.371853,0.737183,4.607415,1.480855,0.673155,1.052538
126,1.745077,0.939893,13.482891,1.786575,0.944896,14.805321,1.857910,0.943226,14.139425,2.415009,0.961783,5.670130,2.385664,0.956175,5.702696,2.105639,0.904288,1.147629


In [68]:
df = df_resultado

In [69]:
final_table = pd.DataFrame({
    'SNR': [df['SNR'].mean(), df['SNR (Wiener)'].mean(), df['SNR (Kalman)'].mean(), df['SNR (Attention Res U-Net)'].mean(), df['SNR (FCRN)'].mean(), df['SNR (PRIDNet)'].mean()],
    'PESQ': [df['PESQ'].mean(), df['PESQ (Wiener)'].mean(), df['PESQ (Kalman)'].mean(), df['PESQ (Attention Res U-Net)'].mean(), df['PESQ (FCRN)'].mean(), df['PESQ (PRIDNet)'].mean()],
    'STOI': [df['STOI'].mean(), df['STOI (Wiener)'].mean(), df['STOI (Kalman)'].mean(), df['STOI (Attention Res U-Net)'].mean(), df['STOI (FCRN)'].mean(), df['STOI (PRIDNet)'].mean()],
}, index=["Ruidoso", "Wiener", "Kalman", "Attention Res U-Net", "FCRN com PESQNet", "PRIDNet"])

In [70]:
final_table

,SNR,PESQ,STOI
Ruidoso,8.680597,1.814686,0.804485
Wiener,9.919522,1.680286,0.769620
Kalman,9.460898,1.901476,0.807711
Attention Res U-Net,10.621112,2.412736,0.852461
FCRN com PESQNet,10.179653,2.259206,0.841510
PRIDNet,3.608414,2.059203,0.774556


In [47]:
# # Criando figuras e eixos separados para cada array
# fig, (ax1, ax2, ax3) = plt.subplots(3, 1)

# # Plotando o primeiro array
# ax1.plot(y_wave_batch[0][-20:])
# ax1.set_ylabel('Sinal de voz ruidoso')

# # Plotando o primeiro array
# ax2.plot(att_batch[0][-20:])
# ax2.set_ylabel('Sinal de voz ruidoso')

# # ax3.plot(s_f)
# # ax3.set_ylabel('Sinal de voz filtrado')

# # Exibindo os gráficos
# plt.show()

In [48]:
from datetime import datetime
# Get the current datetime
current_datetime = datetime.now()

# Format the datetime as a string to use in the file name
datetime_str = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# Define the file name with the datetime
file_name = f"compilado-v3_{datetime_str}.xlsx"

In [34]:
df_resultado[df_resultado['SNR'] != np.inf].to_excel(file_name, index=False)